In [1]:
from azureml.core import Workspace
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import Webservice, AksWebservice
from azureml.core.model import Model


Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.scriptrun = azureml.core.script_run:ScriptRun._from_run_dto with exception (cryptography 36.0.0 (/Users/axelsirota/repos/deploying-managing-models-azure/.venv/lib/python3.7/site-packages), Requirement.parse('cryptography!=1.9,!=2.0.*,!=2.1.*,!=2.2.*,<4.0.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (cryptography 36.0.0 (/Users/axelsirota/repos/deploying-managing-models-azure/.venv/lib/python3.7/site-packages), Requirement.parse('cryptography!=1.9,!=2.0.*,!=2.1.*,!=2.2.*,<4.0.0'), {'azureml-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (cryptography 36.0.0 (/Users/axelsirota/repos/deploying-managing-models-azure/.venv/lib/python3.7/site-packages), Requirement.parse('cryptography

In [2]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

basic-ws-2
test
eastus
790907ce-38df-431c-9fce-11b9ec68781d


In [3]:


#Register the model
from azureml.core.model import Model
model = Model.register(model_path = "sklearn_regression_model.pkl", # this points to a local file
                       model_name = "profiled_model", # this is the name the model is registered as
                       tags = {'area': "diabetes", 'type': "regression"},
                       description = "Ridge regression model to predict diabetes",
                       workspace = ws)

print(model.name, model.description, model.version)



Registering model profiled_model
profiled_model Ridge regression model to predict diabetes 6


In [4]:
%%writefile "requirements_env.txt"

azureml-defaults
azureml-core
inference-schema[numpy-support]
numpy
scikit-learn==0.24.1
scipy
joblib

Overwriting requirements_env.txt


In [5]:
from azureml.core import Environment

environment=Environment.from_pip_requirements('prof-environment', file_path="requirements_env.txt")
environment.register(ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20211124.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "prof-environment",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-forg

In [6]:
%%writefile score.py
# import os
# import pickle
# import json
# import numpy
# from sklearn.externals import joblib
# from sklearn.linear_model import Ridge

def init():
    print("Hello!")
    # global model
    # # AZUREML_MODEL_DIR is an environment variable created during deployment.
    # # It is the path to the model folder (./azureml-models/$MODEL_NAME/$VERSION)
    # # For multiple models, it points to the folder containing all deployed models (./azureml-models)
    # model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'sklearn_regression_model.pkl')
    # # deserialize the model file back into a sklearn model
    # model = joblib.load(model_path)

# note you can pass in multiple rows for scoring
def run(raw_data):
    return ["2"]
    # try:
    #     data = json.loads(raw_data)['data']
    #     data = numpy.array(data)
    #     result = model.predict(data)
    #     # you can return any data type as long as it is JSON-serializable
    #     return result.tolist()
    # except Exception as e:
    #     error = str(e)
    #     return error


Overwriting score.py


In [7]:


from azureml.core.model import InferenceConfig

inf_config = InferenceConfig(entry_script='score.py', environment=environment)



In [8]:
import json
from azureml.core import Datastore
from azureml.core.dataset import Dataset
from azureml.data import dataset_type_definitions

dataset_name='sample_request_data'

dataset_registered = False
try:
    sample_request_data = Dataset.get_by_name(workspace = ws, name = dataset_name)
    dataset_registered = True
except:
    print("The dataset {} is not registered in workspace yet.".format(dataset_name))

if not dataset_registered:
    input_json = {'data': [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
                        [10, 9, 8, 7, 6, 5, 4, 3, 2, 1]]}
    # create a string that can be put in the body of the request
    serialized_input_json = json.dumps(input_json)
    dataset_content = []
    for i in range(100):
        dataset_content.append(serialized_input_json)
    sample_request_data = '\n'.join(dataset_content)
    file_name = "{}.txt".format(dataset_name)
    f = open(file_name, 'w')
    f.write(sample_request_data)
    f.close()

    # upload the txt file created above to the Datastore and create a dataset from it
    data_store = Datastore.get_default(ws)
    data_store.upload_files(['./' + file_name], target_path='sample_request_data')
    datastore_path = [(data_store, 'sample_request_data' +'/' + file_name)]
    sample_request_data = Dataset.Tabular.from_delimited_files(
        datastore_path,
        separator='\n',
        infer_column_types=True,
        header=dataset_type_definitions.PromoteHeadersBehavior.NO_HEADERS)
    sample_request_data = sample_request_data.register(workspace=ws,
                                                    name=dataset_name,
                                                    create_new_version=True)

In [10]:
from datetime import datetime

profile = Model.profile(ws,
            f"profile{datetime.now().strftime('%m%d%Y-%H%M%S')}"[:22],
            [model],
            inf_config,
            input_dataset=sample_request_data)

# profiling is a long running operation and may take up to 25 min
profile.wait_for_completion(show_output=True)
profiling_results = profile.get_results()
print(profiling_results)

Running........................................
Failed


/Users/axelsirota/repos/deploying-managing-models-azure/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:10: UserWarning: Model Profiling operation failed with the following error: Model service has failed with status: CrashLoopBackOff: Back-off restarting failed. This may be caused by errors in your scoring file's init() function. Error logs URL: Log upload failed. Request ID: c72a62c9-2851-4736-a722-dc40416433fc. Inspect ModelProfile.error property for more information.
  # Remove the CWD from sys.path while we load stuff.


AttributeError: 'ModelProfile' object has no attribute 'get_results'